<a href="https://colab.research.google.com/github/HARSHGit45/vidya---GPT/blob/main/MOdel_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling


In [ ]:
with open('train_datap.json', 'r') as f:
    data = json.load(f)

In [ ]:
train_data = []
for item in data:
    prompt = item['prompt']
    for output in item['expected_output']:
        train_data.append(f"Prompt: {prompt}\nQuestion: {output}")


In [ ]:
with open('train_data.txt', 'w', encoding='utf-8') as f:
    for line in train_data:
        f.write(f"{line}\n\n\n")

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="train_data.txt",
    block_size=128
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.216900
1000,0.167000
1500,0.124700
2000,0.099900


TrainOutput(global_step=2320, training_loss=0.14333701627007847, metrics={'train_runtime': 289.1285, 'train_samples_per_second': 16.048, 'train_steps_per_second': 8.024, 'total_flos': 303098757120000.0, 'train_loss': 0.14333701627007847, 'epoch': 10.0})

In [ ]:
trainer.save_model("./trained_model")
tokenizer.save_pretrained("./trained_model")

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("./trained_model")
tokenizer = GPT2Tokenizer.from_pretrained("./trained_model")

In [ ]:
def generate_questions(prompt, num_questions=1):
    input_text = f"Prompt: {prompt}\nQuestion:"
    inputs = tokenizer.encode(input_text, return_tensors='pt')

    # Set pad_token_id to eos_token_id to avoid the warning
    pad_token_id = tokenizer.eos_token_id
    outputs = model.generate(
        inputs,
        max_length=100,
        num_return_sequences=num_questions,
        num_beams=5,
        pad_token_id=pad_token_id
    )

    # Decode the generated outputs
    questions = []
    for output in outputs:
        decoded_output = tokenizer.decode(output, skip_special_tokens=True)
        # Ensure the response is split correctly and only the question part is extracted
        if "Question:" in decoded_output:
            question_part = decoded_output.split("Question:")[1].strip()
            questions.append(question_part)

    return questions

# Example usage
prompt = "Generate questions based on Effects of Electric Current chapter for 1 marks"
questions = generate_questions(prompt, num_questions=1)
for question in questions:
    print(question)  # Print the question directly without any heading


Find odd man out: Voltmeter, ammeter, galvanometer, thermometer.
Prompt: Generate questions based on Effects of Electric Current chapter for 1 marks


In [ ]:
!zip -r trained_model.zip trained_model

  adding: trained_model/ (stored 0%)
  adding: trained_model/generation_config.json (deflated 24%)
  adding: trained_model/model.safetensors (deflated 7%)
  adding: trained_model/special_tokens_map.json (deflated 74%)
  adding: trained_model/vocab.json (deflated 68%)
  adding: trained_model/training_args.bin (deflated 52%)
  adding: trained_model/merges.txt (deflated 53%)
  adding: trained_model/tokenizer_config.json (deflated 54%)
  adding: trained_model/config.json (deflated 52%)


In [ ]:

def generate_questions(prompt, num_questions=1):
    input_text = f"Prompt: {prompt}\nQuestion:"
    inputs = tokenizer.encode(input_text, return_tensors='pt')

    # Set pad_token_id to eos_token_id to avoid the warning
    pad_token_id = tokenizer.eos_token_id
    outputs = model.generate(inputs, max_length=100, num_return_sequences=num_questions, num_beams=5, pad_token_id=pad_token_id)

    # Decode the generated outputs
    questions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    # Truncate to extract only the required question part
    questions = [q.split("Question:")[1].strip() if "Question:" in q else q for q in questions]

    return questions

# Example usage
prompt = "Give reference for What is a redox reaction? Explain with one example. question from Chemical Reactions and Equations chapter"
questions = generate_questions(prompt, num_questions=1)
for i, question in enumerate(questions):
    print(question)  # Print the question directly without any heading

https://www.shaalaa.com/question-bank-solutions/explain-redox-reaction-give-one-example-oxidation-reduction-and-redox-reactions_1364#ref=qp&id=11926
Prompt: Give reference for What is a redox
